<a href="https://colab.research.google.com/github/pinklore/My-Notebook/blob/main/ipynb/ImageLinkConverter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LINK CONVERTER

# **OPTION 1:**

In [ ]:
import re
from IPython.display import HTML, display

def to_direct_view(link):
    m = re.search(r'/d/([^/]+)/', link) or re.search(r'[?&]id=([^&]+)', link)
    if not m:
        raise ValueError("No file ID found in the link.")
    fid = m.group(1)
    return f"https://drive.google.com/uc?export=view&id={fid}"

# 1) Put your share link here
share_link = "https://drive.google.com/file/d/14YSTLVPQEzZ8ekEZyHHdJCSWXwaynZHY/view?usp=drive_link"

# 2) Convert
direct = to_direct_view(share_link)
html_snippet = f'<img src="{direct}" width="600">'

# 3) Render a copy-friendly UI
display(HTML(f"""
<div style="font-family:system-ui,Segoe UI,Arial; line-height:1.45; max-width:900px">
  <div><b>Direct link</b></div>
  <div><a href="{direct}" target="_blank">{direct}</a></div>

  <div style="margin-top:10px"><b>HTML &lt;img&gt; snippet (paste this in a Text cell)</b></div>
  <textarea id="img_snip" rows="3" style="width:100%; font-family:monospace">{html_snippet}</textarea>
  <button id="copy_btn" style="margin-top:6px; padding:6px 10px"
    onclick="navigator.clipboard.writeText(document.getElementById('img_snip').value)
             .then(()=>{{document.getElementById('copied').textContent='Copied!';}})
             .catch(e=>{{document.getElementById('copied').textContent='Copy failed';}})">
    Copy
  </button>
  <span id="copied" style="margin-left:8px; color: #2b7a0b;"></span>

  <div style="margin-top:14px"><b>Preview</b></div>
  <div><img src="{direct}" width="400"></div>
</div>
"""))


**Result from previous cell:**
<img src="https://drive.google.com/uc?export=view&id=14YSTLVPQEzZ8ekEZyHHdJCSWXwaynZHY" width="600">

# **OPTION 2 (BETTER OPTION):**

In [ ]:
import re
from IPython.display import HTML, display

def to_direct_view(link_or_id):
    s = link_or_id.strip()
    m = (re.search(r'/d/([^/]+)/', s) or
         re.search(r'[?&]id=([^&]+)', s) or
         re.fullmatch(r'[A-Za-z0-9_-]{20,}', s))
    if not m:
        raise ValueError("No file ID found.")
    fid = m.group(1) if m.lastindex else m.group(0)
    return f"https://drive.google.com/uc?export=view&id={fid}"

raw = input("Paste one or more links/IDs separated by commas: ").strip()
links = [x.strip() for x in raw.split(",") if x.strip()]

for i in range(0, len(links)):  # traditional index loop
    direct = to_direct_view(links[i])
    snippet = f'<img src="{direct}" width="600">'
    display(HTML(f"""
    <div style="font-family:system-ui,Segoe UI,Arial; line-height:1.45; max-width:900px; margin:4px 0 18px; border-bottom:1px solid #eee; padding-bottom:12px">
      <div>Item {i}</div>
      <div><a href="{direct}" target="_blank">{direct}</a></div>

      <div style="margin-top:8px">Paste this in a Text cell</div>
      <textarea id="img_snip_{i}" rows="2" style="width:100%; font-family:monospace">{snippet}</textarea>
      <button style="margin-top:6px; padding:6px 10px"
        onclick="navigator.clipboard.writeText(document.getElementById('img_snip_{i}').value)
                 .then(()=>{{document.getElementById('copied_{i}').textContent='Copied!';}})
                 .catch(()=>{{document.getElementById('copied_{i}').textContent='Copy failed';}})">
        Copy
      </button>
      <span id="copied_{i}" style="margin-left:8px; color:#2b7a0b;"></span>

      <div style="margin-top:10px"><img src="{direct}" width="420"></div>
    </div>
    """))


Paste one or more links/IDs separated by commas: https://drive.google.com/file/d/1Qt-WyLPuPAR1q-0mBdtRIPnpZQlClgnD/view?usp=drive_link


**Result from previous cell:**

Enter given link in this box:
<img src="https://drive.google.com/uc?export=view&id=1Qt-WyLPuPAR1q-0mBdtRIPnpZQlClgnD" width="600">

# **OPTION 3**

In [ ]:
import re, os, html
from IPython.display import HTML, display
from google.colab import output

def to_direct_view(link_or_id):
    s = link_or_id.strip()
    m = (re.search(r'/d/([^/]+)/', s) or
         re.search(r'[?&]id=([^&]+)', s) or
         re.fullmatch(r'[A-Za-z0-9_-]{20,}', s))
    if not m:
        raise ValueError("No file ID found.")
    fid = m.group(1) if m.lastindex else m.group(0)
    return f"https://drive.google.com/uc?export=view&id={fid}"

def build_table_html(direct_urls, labels=None, width_px=600, out_path="/content/last_img_table.html"):
    if labels is None or len(labels) != len(direct_urls):
        labels = [f"Item {i}" for i in range(0, len(direct_urls))]
    head = f"""<!DOCTYPE html>
<html><head><meta charset="utf-8"><style>
  body {{ font-family: system-ui, Segoe UI, Arial, sans-serif; margin: 6px; }}
  table {{ border-collapse: collapse; width: 100%; }}
  th, td {{ border: 1px solid #ddd; padding: 6px; text-align: left; vertical-align: top; }}
  th {{ background: #f6f6f6; }}
  img {{ max-width: 100%; height: auto; }}
  .wrap {{ max-width: {width_px}px; }}
  a {{ word-break: break-all; }}
</style></head><body>
<table>
<thead><tr><th>#</th><th>Label</th><th>Direct link</th><th>Preview</th></tr></thead><tbody>
"""
    rows = []
    for i in range(0, len(direct_urls)):
        url = direct_urls[i]
        lab = html.escape(str(labels[i]))
        rows.append(f"""
<tr>
  <td>{i}</td>
  <td>{lab}</td>
  <td><a href="{url}" target="_blank">{url}</a></td>
  <td class="wrap"><img src="{url}" width="{width_px}"></td>
</tr>
""")
    tail = "</tbody></table></body></html>"
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(head + "".join(rows) + tail)
    return out_path

def _cb_build_table(links_str, labels_str, width_px):
    links = [x.strip() for x in links_str.split(",") if x.strip()]
    labels = [x.strip() for x in labels_str.split(",")] if labels_str.strip() else None
    directs = []
    for i in range(0, len(links)):
        directs.append(to_direct_view(links[i]))
    table_path = build_table_html(directs, labels=labels, width_px=width_px, out_path="/content/last_img_table.html")
    h = min(240 + 260 * len(directs), 1800)
    iframe_html = f'<iframe src="/content/last_img_table.html" width="980" height="{h}" style="border:0"></iframe>'
    return iframe_html



output.register_callback('build_table_cb', _cb_build_table)

display(HTML("""
<div style="font-family:system-ui,Segoe UI,Arial; line-height:1.45; max-width:980px">
  <div><b>Links/IDs (comma-separated)</b></div>
  <textarea id="links" rows="3" style="width:100%"> </textarea>

  <div style="margin-top:8px"><b>Labels (optional, comma-separated)</b></div>
  <input id="labels" style="width:100%" placeholder="e.g., Intro, Result, Diagram">

  <div style="margin-top:8px"><b>Image width (px)</b></div>
  <input id="w" type="number" value="600" style="width:120px">

  <div style="margin-top:10px">
    <button onclick="(async () => {
      const links = document.getElementById('links').value;
      const labels = document.getElementById('labels').value;
      const w = parseInt(document.getElementById('w').value || '600');
      const r = await google.colab.kernel.invokeFunction('build_table_cb', [links, labels, w], {});
      const txt = r['data']['text/plain'] || '';
      const iframe = txt.substring(1, txt.length-1); // strip quotes from repr
      document.getElementById('out').value = iframe;
      document.getElementById('preview').innerHTML = iframe;
    })()" style="padding:6px 12px">Generate</button>
  </div>

  <div style="margin-top:10px"><b>Paste this in a Text cell once</b></div>
  <textarea id="out" rows="2" style="width:100%; font-family:monospace"></textarea>

  <div style="margin-top:14px"><b>Live preview</b></div>
  <div id="preview" style="border:1px solid #ddd; padding:6px"></div>
</div>
"""))

